In [31]:
import os
import json
import pandas as pd
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import faiss


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
pip install faiss-cpu



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-dotenv

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Cushion

In [33]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# text analysis

import os
import pandas as pd
from openai import OpenAI

# Your OpenAI / Databricks key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# Load your file
df = pd.read_csv("cushion_reviews.csv")   # or read_csv("your_file.csv")
reviews = df["review"].tolist()        # make sure your column is named "review"

# Combine all reviews into ONE document
all_reviews_text = "\n\n".join(reviews)

prompt = f"""
You are analyzing a collection of customer product reviews.

Using ALL the reviews below, provide a holistic aggregated analysis:

1. Overall sentiment trend across all reviews (positive / negative / mixed)
2. The most common aspects customers say need improvement
3. The most valuable features customers appreciate
4. A 3–5 sentence summary of the overall customer perception

Only report issues proportional to their frequency. Avoid overstating minority opinions.

Return results in this JSON format:

{{
  "overall_sentiment": "",
  "top_improvement_aspects": [],
  "top_valuable_features": [],
  "summary": ""
}}

ALL REVIEWS:
\"\"\"{all_reviews_text}\"\"\"
"""

response = client.chat.completions.create(
    model="gpt-4.1-mini",   # or gpt-4.1, gpt-5, Databricks endpoint
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

print(response.choices[0].message.content)

{
  "overall_sentiment": "mixed",
  "top_improvement_aspects": [
    "shade range and color matching",
    "product quantity/size (small amount of foundation)",
    "cakey/thick texture with heavier application",
    "compatibility with primers and other makeup products",
    "packaging issues (damaged or used products received)"
  ],
  "top_valuable_features": [
    "long-lasting wear and longevity",
    "natural, lightweight, and smooth finish",
    "good coverage with buildable formula",
    "high-quality, sturdy, and attractive packaging",
    "ease of application with included applicator"
  ],
  "summary": "Customer perception of this Korean cushion foundation is generally positive regarding its long-lasting wear, natural and lightweight finish, and good coverage that can be built up without feeling heavy. Many appreciate the high-quality and attractive packaging as well as the ease of application. However, there are notable concerns about the limited and inconsistent shade range,

In [38]:
import os
import faiss
import numpy as np
import pandas as pd
from openai import OpenAI

# Your OpenAI / Databricks key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# -----------------------------
# Load your reviews dataset
# -----------------------------
# Load your file
df = pd.read_csv("cushion_reviews.csv")   # or read_csv("your_file.csv")
reviews = df["review"].tolist()    


# -----------------------------
# Create chunks for RAG
# -----------------------------
def chunk_text(reviews, chunk_size=500):
    chunks = []
    buffer = ""
    for review in reviews:
        if len(buffer) + len(review) < chunk_size:
            buffer += " " + review
        else:
            chunks.append(buffer.strip())
            buffer = review
    if buffer:
        chunks.append(buffer.strip())
    return chunks

chunks = chunk_text(reviews)


# -----------------------------
# Step 1: Embed chunks
# -----------------------------
def embed_text(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return np.array([item.embedding for item in response.data])

embeddings = embed_text(chunks)


# -----------------------------
# Step 2: Store in FAISS index
# -----------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)


# -----------------------------
# Step 3: Retrieve top-K relevant chunks
# -----------------------------
def retrieve(query, k=5):
    query_embedding = embed_text([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]


# -----------------------------
# Step 4: LLM Synthesis (the actual RAG step)
# -----------------------------
def rag_analyze(query):
    retrieved_context = retrieve(query, k=5)
    context = "\n\n".join(retrieved_context)

    prompt = f"""
You are analyzing customer product reviews.

Using ONLY the information in the retrieved context, answer:

1. Overall sentiment trend
2. Common aspects needing improvement
3. Features customers value most
4. A 3-5 sentence summary

CONTEXT:
\"\"\"{context}\"\"\"

QUESTION:
\"\"\"{query}\"\"\"
"""

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content


# -----------------------------
# Run RAG query
# -----------------------------
query = "What is the overall sentiment, improvement areas, and valuable features of this product?"
result = rag_analyze(query)

print(result)


1. **Overall sentiment trend:**  
The overall sentiment is highly positive. Reviewers consistently praise the product for its packaging, lightweight feel, buildable coverage, shade range, and long-lasting wear.

2. **Common aspects needing improvement:**  
The main area needing improvement is the potential for the foundation to oxidize and turn orange, especially for those seeking cool undertones. Some users also noted that if applied too heavily, it can settle into creases or lines.

3. **Features customers value most:**  
Customers value the luxurious and practical packaging, inclusive shade range, lightweight and comfortable formula, buildable and natural-looking coverage, long-lasting wear, and smooth, radiant finish.

4. **Summary:**  
Customers are very pleased with this cushion foundation, highlighting its high-end packaging, inclusive shade range, and lightweight, buildable formula that provides a natural yet long-lasting finish. The foundation is praised for being easy to appl

## Suncreen

In [7]:
pip install requests



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import re

# Read the original file
with open("reviews_fixed.json", "r", encoding="utf-8", errors="ignore") as f:
    text = f.read().strip()

# Add commas between objects and wrap them into an array
fixed_json = "[" + re.sub(r"}\s*{", "},{", text) + "]"

# Save as a new file
with open("reviews_array.json", "w", encoding="utf-8") as f:
    f.write(fixed_json)

print("✅ File converted to valid JSON array format: reviews_array.json")


✅ File converted to valid JSON array format: reviews_array.json


In [25]:
import json
import pandas as pd

# Load your full JSON file
with open("reviews_array.json", "r", encoding="utf-8") as f:
    pages = json.load(f)

all_reviews = []
for p in pages:
    for r in p.get("reviews", []):
        all_reviews.append({
            "page": p.get("page"),
            "review_id": r.get("id"),
            "date": r.get("date"),
            "author_name": r.get("author_name"),
            "rating": r.get("rating"),
            "review_title": r.get("review_title"),
            "review_text": r.get("review_text"),
            "verified_purchase": r.get("meta_data", {}).get("verified_purchase"),
            "helpful_votes": r.get("meta_data", {}).get("helpful_vote_count"),
            "is_vine_voice": r.get("meta_data", {}).get("is_vine_voice"),
            "location": r.get("location"),
        })

df = pd.DataFrame(all_reviews)
df.to_csv("amazon_reviews_full.csv", index=False, encoding="utf-8-sig")

print(f"✅ Extracted {len(df)} reviews from {len(pages)} pages.")


✅ Extracted 100 reviews from 10 pages.


In [ ]:
# text analysis

import os
import pandas as pd
from openai import OpenAI

# Your OpenAI / Databricks key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# Load your file
df = pd.read_csv("amazon_reviews_full.csv")   # or read_csv("your_file.csv")
reviews = df["review_text"].tolist()        # make sure your column is named "review_text"

# Combine all reviews into ONE document
all_reviews_text = "\n\n".join(reviews)

prompt = f"""
You are analyzing a collection of customer product reviews.

Using ALL the reviews below, provide a holistic aggregated analysis:

1. Overall sentiment trend across all reviews (positive / negative / mixed)
2. The most common aspects customers say need improvement
3. The most valuable features customers appreciate
4. A 3–5 sentence summary of the overall customer perception

Describe performance only in the context reviewers specify (e.g., skin type, environment, layering). Do not generalize.

Return results in this JSON format:

{{
  "overall_sentiment": "",
  "top_improvement_aspects": [],
  "top_valuable_features": [],
  "summary": ""
}}

ALL REVIEWS:
\"\"\"{all_reviews_text}\"\"\"
"""

response = client.chat.completions.create(
    model="gpt-4.1-mini",   # or gpt-4.1, gpt-5, Databricks endpoint
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

print(response.choices[0].message.content)

{
  "overall_sentiment": "mixed",
  "top_improvement_aspects": [
    "eye irritation and stinging when sunscreen gets into eyes",
    "greasy or oily finish, especially for oily or combination skin types",
    "strong or unpleasant scent/fragrance for some users",
    "thick texture and difficulty blending for some users",
    "small bottle size and price increase over time"
  ],
  "top_valuable_features": [
    "no white cast, suitable for all skin tones including darker skin",
    "hydrating and moisturizing effect, good for dry and sensitive skin",
    "lightweight feel that works well under makeup for many users",
    "high SPF 50 protection with added skincare benefits like Vitamin E",
    "pleasant or mild scent compared to typical sunscreens",
    "non-comedogenic and generally non-irritating for sensitive skin"
  ],
  "summary": "Customer perception of the COSRX Vitamin E Vitalizing Sunscreen SPF 50 is generally positive, especially praising its lack of white cast, hydrating pr

In [47]:
import os
import faiss
import numpy as np
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time

# -----------------------------
# Setup OpenAI client
# -----------------------------
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("❌ OPENAI_API_KEY not found. Check your .env file.")
client = OpenAI(api_key=OPENAI_API_KEY)

# -----------------------------
# Step 1: Load & Clean Data
# -----------------------------
df = pd.read_csv("amazon_reviews_full.csv")

# Keep only valid, non-empty text reviews
reviews = df["review_text"].dropna().astype(str)
reviews = [r.strip() for r in reviews if len(r.strip()) > 5]

print(f"✅ Loaded {len(reviews)} valid reviews")

# -----------------------------
# Step 2: Chunk Reviews
# -----------------------------
def chunk_text(reviews, chunk_size=800):
    chunks, buffer = [], ""
    for review in reviews:
        if len(buffer) + len(review) < chunk_size:
            buffer += " " + review
        else:
            chunks.append(buffer.strip())
            buffer = review
    if buffer:
        chunks.append(buffer.strip())
    return chunks

chunks = chunk_text(reviews)
print(f"✅ Created {len(chunks)} chunks for embedding")

# -----------------------------
# Step 3: Embed Chunks
# -----------------------------
def embed_text(texts, model="text-embedding-3-large", batch_size=50, sleep_time=2):
    """
    Batch-embed text safely with rate-limit handling
    """
    if isinstance(texts, str):
        texts = [texts]
    elif not isinstance(texts, list):
        raise ValueError("Input must be a list or string")

    texts = [str(t).strip() for t in texts if t and isinstance(t, (str, bytes)) and len(str(t).strip()) > 0]
    if not texts:
        raise ValueError("No valid string input to embed.")

    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="🔹 Embedding chunks"):
        batch = texts[i:i+batch_size]
        try:
            response = client.embeddings.create(model=model, input=batch)
            all_embeddings.extend([d.embedding for d in response.data])
        except Exception as e:
            print(f"⚠️ API Error on batch {i // batch_size}: {e}")
            time.sleep(sleep_time * 2)
            continue
        time.sleep(sleep_time)
    return np.array(all_embeddings)

embeddings = embed_text(chunks)
print(f"✅ Embedded {len(embeddings)} chunks successfully")

# -----------------------------
# Step 4: Store in FAISS Index
# -----------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save FAISS and metadata
faiss.write_index(index, "sunscreen_index.faiss")
pd.DataFrame({"chunk_id": range(len(chunks)), "text": chunks}).to_csv("sunscreen_chunks.csv", index=False)

print("✅ FAISS index saved as 'sunscreen_index.faiss'")
print("✅ Metadata saved as 'sunscreen_chunks.csv'")

# -----------------------------
# Step 5: Retrieve Function
# -----------------------------
def retrieve(query, k=5):
    query_embedding = embed_text([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

# -----------------------------
# Step 6: RAG-style LLM Analysis
# -----------------------------
def rag_analyze(query):
    retrieved_context = retrieve(query, k=5)
    context = "\n\n".join(retrieved_context)

    prompt = f"""
You are analyzing customer product reviews.

Using ONLY the information in the retrieved context, answer:

1. Overall sentiment trend
2. Common aspects needing improvement
3. Features customers value most
4. A 3–5 sentence summary

CONTEXT:
\"\"\"{context}\"\"\"

QUESTION:
\"\"\"{query}\"\"\"
"""

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content

# -----------------------------
# Step 7: Run Query
# -----------------------------
query = "What is the overall sentiment, improvement areas, and valuable features of this product?"
result = rag_analyze(query)
print("\n🧠 RAG Result:\n")
print(result)


✅ Loaded 99 valid reviews
✅ Created 49 chunks for embedding


🔹 Embedding chunks: 100%|██████████| 1/1 [00:03<00:00,  3.90s/it]


✅ Embedded 48 chunks successfully
✅ FAISS index saved as 'sunscreen_index.faiss'
✅ Metadata saved as 'sunscreen_chunks.csv'


🔹 Embedding chunks: 100%|██████████| 1/1 [00:03<00:00,  3.48s/it]



🧠 RAG Result:

1. **Overall sentiment trend:**  
The overall sentiment is very positive. Most reviewers express high satisfaction with the sunscreen, praising its effectiveness, texture, and lack of irritation. There are a few minor criticisms, but the general tone is enthusiastic and appreciative.

2. **Common aspects needing improvement:**  
- Some users find the texture a bit thick and difficult to blend, especially for medium to deeper skin tones.
- A slight white cast is noted by some, particularly on certain skin tones.
- The product can feel heavy after a few hours, which may not suit oily or combination skin.
- One user wishes the tube was bigger.
- There was an isolated issue with packaging leakage during shipping.

3. **Features customers value most:**  
- No white cast or residue for most users
- Lightweight feel and easy absorption
- High SPF 50 protection
- Hydrating and non-irritating, even for sensitive skin
- Works well under makeup
- Good customer service
- Pleasant s